In [ ]:
! pip install -Uqq fastai

In [ ]:
## Todo: how to plot distribution training set, test set
## Resnet type ?
## Preprocessing ???


In [ ]:
from fastai.tabular.all import *
from sklearn.model_selection import StratifiedKFold, KFold


In [ ]:
path = '/kaggle/input/tabular-playground-series-nov-2021/train.csv'
df_train = pd.read_csv(path)
df_train.head()
idx = list(range(len(df_train)))
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv')

In [ ]:
cont_names = list(df_train.drop(['id', 'target', 'f2', 'f35', 'f44'], axis = 1).columns)

In [ ]:
# for col in cont_names:
#     df_train[col] = df_train[col].apply(lambda x: 1 if np.cbrt(x) > 0 else 0)
#     df_test[col] = df_test[col].apply(lambda x: 1 if np.cbrt(x) > 0 else 0)

In [ ]:
splits = RandomSplitter(valid_pct=0.1)(range_of(df_train))
to = TabularPandas(df_train, procs=[FillMissing,Normalize],
                   y_block = CategoryBlock(),
                   cont_names = cont_names,
                   y_names='target',
                   splits=splits)

dls = to.dataloaders(bs=2048)
dls.show_batch()

In [ ]:
# dims = [len(cont_names), 256, 512, 256, 100, 64]

In [ ]:
dims = [len(cont_names), 100, 50]

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layers = nn.ModuleList([LinBnDrop(n_in,n_out, act=nn.SiLU(), p=.1, lin_first=True) for n_in, n_out in zip(dims, dims[1:])])
        self.final = LinBnDrop(dims[-1],2, lin_first=True, bn=False)
    
    def forward(self,*x):
        x = x[1] # take into accound just cont variables
        for layer in self.layers:
            x = layer(x)
        x = self.final(x)
        return x
        

In [ ]:
# learn = Learner(dls, model, metrics=RocAucBinary())
# model = Model()
# learn = tabular_learner(dls, metrics=accuracy, config=config, layers = [100,50])
# learn.lr_find()
# cbs=[EarlyStoppingCallback(monitor='roc_auc_score', patience=8), 
#       ReduceLROnPlateau(monitor='valid_loss', patience=8), 
#       SaveModelCallback(monitor='roc_auc_score')]
# learn.fit_one_cycle(30, lr_max=2e-3, cbs=cbs)

In [ ]:
# splits = RandomSplitter(valid_pct=0.1)(range_of(df_train))
# to = TabularPandas(df_train, procs=[Categorify, FillMissing,Normalize],
#                    y_block = CategoryBlock(),
#                    cont_names = cont_names,
#                    y_names='target',
#                    splits=splits)

# dls = to.dataloaders(bs=1024)
# dls.show_batch()

# learn = tabular_learner(dls, metrics=accuracy, config=config, layers = [100,50])
# learn.lr_find()
# learn.fit_one_cycle(20, lr_max=2e-3, cbs=[EarlyStoppingCallback(monitor='accuracy', patience=10), ReduceLROnPlateau(monitor='valid_loss', patience=5), SaveModelCallback(monitor='accuracy')])

In [ ]:
# dl = learn.dls.test_dl(df_test)
# preds = learn.get_preds(dl=dl)
# results = preds[0].argmax(axis=1)
# results = results.tolist()

In [ ]:
kf = KFold(n_splits=10, shuffle=True)
preds_fold = []
cbs=[EarlyStoppingCallback(monitor='roc_auc_score', patience=8), 
      ReduceLROnPlateau(monitor='valid_loss', patience=8), 
      SaveModelCallback(monitor='roc_auc_score')]
for fold, (train_index, valid_index) in enumerate(kf.split(idx)):
    print(f'Fold {fold}')
    to = TabularPandas(df_train, procs=[FillMissing,Normalize],
                       y_block = CategoryBlock(),
                       cont_names = cont_names,
                       y_names='target',
                       splits=(list(train_index), list(valid_index)))
    dls = to.dataloaders(bs=2048)
    model = Model()
    learn = Learner(dls, model, metrics=RocAucBinary())
#     learn = tabular_learner(dls, metrics=accuracy, config=config, layers = [100,50])
    learn.fit_one_cycle(10, lr_max=2e-3, cbs=cbs)
    dl = learn.dls.test_dl(df_test)
    preds = learn.get_preds(dl=dl)
#     results = preds[0].argmax(axis=1)
#     results = results.tolist()
    results = preds[0].numpy()
    preds_fold.append(results)

In [ ]:
final_preds = np.argmax(np.mean(np.array(preds_fold), axis=0), axis=1)

In [ ]:
submission = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv')

In [ ]:
submission['target'] = final_preds

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)